In [1]:
import numpy as np
import pandas as pd
import math
import random
import subprocess

from multiprocessing.pool import ThreadPool as Pool

## Шифр Хейса

In [2]:
n=4
r=6
n2=n*n

s_box=[1, 2, 3, 14, 6, 13, 11, 8, 15, 10, 12, 5, 7, 9, 0, 4]

In [3]:
class Plus_key:
    def result(self, x, key):
        return x^key

In [4]:
class S_box:
    def __init__(self, bits, s_box = None ):
        self.bits = bits
        self.s_box = s_box
    
    def result(self, x):
        if self.bits==n:
            return self.s_box[x]
        if self.bits==n2:
            s=0
            for i in range(n):
                x2=(x&15)
                s += (self.s_box[x2]<<4*i)
                x = x>>4
        return s
    
    def reverse(self, x):
        if self.bits==n:
            return self.s_box.index(x)
        if self.bits==n2:
            s=0
            for i in range(n):
                x2=(x&15)
                s += (self.s_box.index(x2)<<4*i)
                x = x>>4
        return s

In [5]:
class Shifting:
    def result(self, x):
        shifted_x=0
        for i in range(n):
            for j in range(n):
                shift = x & (1<<(4*i+j))
                if shift != 0:
                    shifted_x += 1 << (4*j+i)
        return shifted_x

In [6]:
def Heys(x, key, printing=False):
    y = x
    pk = Plus_key()
    s = S_box(n2, s_box)
    shift = Shifting()
    for i in range(r):
        y = pk.result(y, key[i])
        y = s.result(y)
        y = shift.result(y)
        if printing:
            print("round : " + str(i+1))
            print("   key = " + str(key[i]))
            print("   y = " + str(y))
    y = pk.result(y, key[r])
    if printing:
        print("round : " + str(7))
        print("   key = " + str(key[r]))
        print("   y = " + str(y))
    return y

In [7]:
key = [0 for i in range(r+1)]
for i in range(r+1):
    key[i] = random.randint(0, 2**16 - 1)
print(key)

[28909, 12201, 47342, 25249, 56784, 17977, 41203]


In [8]:
x = 1

In [9]:
Heys(x, key, True)

round : 1
   key = 28909
   y = 33045
round : 2
   key = 12201
   y = 35603
round : 3
   key = 47342
   y = 57025
round : 4
   key = 25249
   y = 11375
round : 5
   key = 56784
   y = 2882
round : 6
   key = 17977
   y = 27013
round : 7
   key = 41203
   y = 51574


51574

## Пошук диференціалів

In [10]:
def tableDF():
    s = S_box(n, s_box)
    DF=[[0 for i in range(n2)] for i in range (n2)]
    DF[0][0]=1
    for alpha in range(1,n2):
        for beta in range(1,n2):
            for x in range(n2):
                sx=s.result(x)
                sx_alpha=s.result(x^alpha)
                if sx^sx_alpha==beta:
                    DF[alpha][beta]+=1
            DF[alpha][beta]/=(n2)
    return DF

In [11]:
DF=tableDF()

In [12]:
a=["A0", "A1", "A2", "A3", "A4", "A5", "A6", "A7", "A8", "A9", "A10", "A11", "A12", "A13", "A14", "A15"]
b=["B0", "B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B9", "B10", "B11", "B12", "B13", "B14", "B15"]
pd.DataFrame(DF, columns = b, index = a)

,B0,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11,B12,B13,B14,B15
A0,1,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
A1,0,0.000,0.000,0.250,0.125,0.125,0.000,0.000,0.000,0.125,0.000,0.125,0.000,0.125,0.125,0.000
A2,0,0.000,0.125,0.125,0.000,0.125,0.000,0.125,0.000,0.000,0.000,0.000,0.125,0.250,0.000,0.125
A3,0,0.125,0.000,0.125,0.000,0.000,0.250,0.000,0.000,0.125,0.125,0.000,0.000,0.000,0.125,0.125
A4,0,0.125,0.000,0.125,0.000,0.000,0.125,0.125,0.250,0.000,0.000,0.000,0.125,0.000,0.000,0.125
A5,0,0.000,0.000,0.000,0.125,0.250,0.125,0.000,0.125,0.000,0.000,0.125,0.125,0.125,0.000,0.000
A6,0,0.000,0.000,0.125,0.000,0.125,0.000,0.000,0.000,0.000,0.250,0.125,0.125,0.000,0.125,0.125
A7,0,0.000,0.125,0.000,0.000,0.125,0.000,0.000,0.125,0.250,0.125,0.125,0.000,0.000,0.125,0.000
A8,0,0.125,0.000,0.000,0.125,0.000,0.000,0.000,0.125,0.000,0.000,0.250,0.125,0.000,0.125,0.125
A9,0,0.000,0.125,0.000,0.000,0.000,0.125,0.000,0.125,0.000,0.125,0.125,0.000,0.125,0.000,0.250


In [13]:
def create_list(alpha):
    a=[0 for i in range(n)]
    Bq=[]
    for i in range(n):
        a[n-i-1]=int(alpha%n2)
        alpha/=n2
    b1=[j for j in range(n2) if DF[a[0]][j]!=0]
    b2=[j for j in range(n2) if DF[a[1]][j]!=0]
    b3=[j for j in range(n2) if DF[a[2]][j]!=0]
    b4=[j for j in range(n2) if DF[a[3]][j]!=0]
    #print(b1)
    #print(b2)
    #print(b3)
    #print(b4)
    for j1 in range(len(b1)):
        for j2 in range(len(b2)):
            for j3 in range(len(b3)):
                for j4 in range(len(b4)):
                    b=b4[j4] + (b3[j3]<<4) + (b2[j2]<<8) + (b1[j1]<<12)
                    q=DF[a[0]][b1[j1]] * DF[a[1]][b2[j2]] * DF[a[2]][b3[j3]] * DF[a[3]][b4[j4]]
                    Bq.append([b,q])
    return Bq

In [15]:
p=0.0007

In [16]:
def DifferentialSearch2(alpha):
    shift = Shifting()
    G0=dict()
    G0[alpha]=1
    beta_dict=dict()
    for t in range(1,r):
        G1=dict()
        for g in G0:
            p_i=G0[g]
            if g not in beta_dict:
                delta_beta=create_list(g)
                beta_dict[g]=delta_beta
            else:
                delta_beta=beta_dict[g]
            #print(delta_beta)
            for j in range(len(delta_beta)):
                if delta_beta[j][0] in G1:
                    G1[delta_beta[j][0]] += p_i*delta_beta[j][1]
                else:
                    G1[delta_beta[j][0]]=p_i*delta_beta[j][1]
            #print(G1)
        G=dict()
        for g1 in G1:
            if G1[g1]>p:
                g = shift.result(g1)
                G[g]=G1[g1]
        #print("G = "+str(G))
        G0=dict(G)
        #print()
        #print()
        #print()
    return G0

In [17]:
diff_search_result2=dict()
for i in range(1,n2):
    for j in range(4):
        alpha=i<<4*j
        G=DifferentialSearch2(alpha)
        if len(G)!=0:
            print(alpha, G)
            diff_search_result2[alpha]=G

1024 {16384: 0.000736236572265625}
16384 {32768: 0.000732421875}
1280 {16452: 0.000820159912109375}
20480 {32904: 0.0007781982421875}
2048 {16452: 0.000751495361328125}
32768 {32904: 0.000732421875}
3328 {16452: 0.00080108642578125}
53248 {32904: 0.00075531005859375}
14 {32904: 0.0009627342224121094}
3584 {68: 0.000885009765625, 16384: 0.000865936279296875, 17408: 0.000759124755859375, 17476: 0.000759124755859375, 1024: 0.00075531005859375, 16452: 0.001216888427734375, 17472: 0.00075531005859375, 4113: 0.0008974075317382812, 32904: 0.00081634521484375}
57344 {8: 0.0007500648498535156, 136: 0.0009560585021972656, 34944: 0.0008416175842285156, 34952: 0.0008416175842285156, 32768: 0.00084686279296875, 34816: 0.000743865966796875, 16452: 0.0008337497711181641, 4113: 0.0008337497711181641, 2048: 0.000743865966796875, 32904: 0.001201629638671875}


In [18]:
diff_result=[]
for alpha in diff_search_result2:
    for beta in diff_search_result2[alpha]:
        prob = diff_search_result2[alpha][beta]
        diff_result.append(((alpha, beta), prob))
sorted_diff_result=sorted(diff_result, key = lambda x: x[1], reverse = True) 

In [19]:
sorted_diff_result

[((3584, 16452), 0.001216888427734375),
 ((57344, 32904), 0.001201629638671875),
 ((14, 32904), 0.0009627342224121094),
 ((57344, 136), 0.0009560585021972656),
 ((3584, 4113), 0.0008974075317382812),
 ((3584, 68), 0.000885009765625),
 ((3584, 16384), 0.000865936279296875),
 ((57344, 32768), 0.00084686279296875),
 ((57344, 34944), 0.0008416175842285156),
 ((57344, 34952), 0.0008416175842285156),
 ((57344, 16452), 0.0008337497711181641),
 ((57344, 4113), 0.0008337497711181641),
 ((1280, 16452), 0.000820159912109375),
 ((3584, 32904), 0.00081634521484375),
 ((3328, 16452), 0.00080108642578125),
 ((20480, 32904), 0.0007781982421875),
 ((3584, 17408), 0.000759124755859375),
 ((3584, 17476), 0.000759124755859375),
 ((53248, 32904), 0.00075531005859375),
 ((3584, 1024), 0.00075531005859375),
 ((3584, 17472), 0.00075531005859375),
 ((2048, 16452), 0.000751495361328125),
 ((57344, 8), 0.0007500648498535156),
 ((57344, 34816), 0.000743865966796875),
 ((57344, 2048), 0.000743865966796875),
 ((102

## Атака (пошук ключів)

In [20]:
key

[28909, 12201, 47342, 25249, 56784, 17977, 41203]

In [21]:
def attack(prob_key, beta, messages):
    result = 0
    for (y1, y2) in messages:
        y_1 = pk.result(y1, prob_key)
        y_2 = pk.result(y2, prob_key)
        y_1 = shift.result(y_1)
        y_2 = shift.result(y_2)
        y_1 = s.reverse(y_1)
        y_2 = s.reverse(y_2)
        if y_1^y_2==beta:
            result += 1
    return (prob_key, result)

In [22]:
keys = [(i,0) for i in range(1, 2**16-1)]

iteration = 1

pk = Plus_key()
s = S_box(n2, s_box)
shift = Shifting()

for (alpha, beta), prob in sorted_diff_result:
    messages = [random.randint(1, 2**16 - 1) for i in range(5000)]
    cipher_messages = [(Heys(m, key), Heys(m^alpha, key)) for m in messages]
    
    for i in range(len(keys)):
        keys[i] = attack(keys[i][0], beta, cipher_messages)
                
    max_prob = max(keys, key = lambda x: x[1])[1]
    keys = [(i, 0) for (i,p) in keys if p == max_prob]
    
    print('Iteration', iteration)
    print('Max frequency', max_prob)
    print('Keys with max frequency', keys)
    print('---------------------------')
    
    iteration +=1
    if len(keys)==1:
        break

Iteration 1
Max frequency 7
Keys with max frequency [(41139, 0), (41143, 0), (41203, 0), (41207, 0), (42163, 0), (42167, 0), (42227, 0), (42231, 0), (57523, 0), (57527, 0), (57587, 0), (57591, 0), (58547, 0), (58551, 0), (58611, 0), (58615, 0)]
---------------------------
Iteration 2
Max frequency 7
Keys with max frequency [(41139, 0), (41143, 0), (41203, 0), (41207, 0), (42163, 0), (42167, 0), (42227, 0), (42231, 0), (57523, 0), (57527, 0), (57587, 0), (57591, 0), (58547, 0), (58551, 0), (58611, 0), (58615, 0)]
---------------------------
Iteration 3
Max frequency 4
Keys with max frequency [(41139, 0), (41143, 0), (41203, 0), (41207, 0), (42163, 0), (42167, 0), (42227, 0), (42231, 0), (57523, 0), (57527, 0), (57587, 0), (57591, 0), (58547, 0), (58551, 0), (58611, 0), (58615, 0)]
---------------------------
Iteration 4
Max frequency 3
Keys with max frequency [(41139, 0), (41143, 0), (41203, 0), (41207, 0), (42163, 0), (42167, 0), (42227, 0), (42231, 0), (57523, 0), (57527, 0), (57587, 

In [23]:
keys

[(41203, 0)]

In [24]:
keys = [(i,0) for i in range(1, 2**16-1)]

iteration = 1

pk = Plus_key()
s = S_box(n2, s_box)
shift = Shifting()

for (alpha, beta), prob in sorted_diff_result:
    messages = [random.randint(1, 2**16 - 1) for i in range(5000)]
    
    with open(f'inputs/input_{iteration}_orig.bin', 'wb') as f:
        for text in messages:
            f.write(text.to_bytes(2, 'little'))
            
    with open(f'inputs/input_{iteration}_mut.bin', 'wb') as f:
        for text in messages:
            f.write((text^alpha).to_bytes(2, 'little'))
    
    subprocess.Popen(f'Heys.exe e 08 ./inputs/input_{iteration}_orig.bin ./outputs/output_{iteration}_orig.bin', stdin = subprocess.PIPE, shell = True).communicate(timeout = 5)
    subprocess.Popen(f'Heys.exe e 08 ./inputs/input_{iteration}_mut.bin ./outputs/output_{iteration}_mut.bin', stdin = subprocess.PIPE, shell = True).communicate(timeout = 5)

    cipher_texts = []

    with open(f'outputs/output_{iteration}_orig.bin', 'rb') as f:
        line = f.read()
        concatenated_cipher_text = int.from_bytes(line, byteorder='little')

        for i in range(len(line) // 2):
            cipher_texts.append((concatenated_cipher_text >> (i * 16)) & (2**16 - 1))
            
    mutated_cipher_texts = []

    with open(f'outputs/output_{iteration}_mut.bin', 'rb') as f:
        line = f.read()
        concatenated_cipher_text = int.from_bytes(line, byteorder='little')

        for i in range(len(line) // 2):
            mutated_cipher_texts.append((concatenated_cipher_text >> (i * 16)) & (2**16 - 1))
    
    cipher_messages = [(cipher_texts[i], mutated_cipher_texts[i]) for i in range(len(cipher_texts))]
    
    for i in range(len(keys)):
        keys[i] = attack(keys[i][0], beta, cipher_messages)
                
    max_prob = max(keys, key = lambda x: x[1])[1]
    keys = [(i, 0) for (i,p) in keys if p == max_prob]
    
    print('Iteration', iteration)
    print('Max frequency', max_prob)
    print('Keys with max frequency', keys)
    print('---------------------------')
    
    iteration +=1
    if len(keys)==1:
        break

Iteration 1
Max frequency 5
Keys with max frequency [(47619, 0), (47623, 0), (47683, 0), (47687, 0), (48643, 0), (48647, 0), (48707, 0), (48711, 0), (64003, 0), (64007, 0), (64067, 0), (64071, 0), (65027, 0), (65031, 0), (65091, 0), (65095, 0)]
---------------------------
Iteration 2
Max frequency 16
Keys with max frequency [(47619, 0), (47623, 0), (47683, 0), (47687, 0), (48643, 0), (48647, 0), (48707, 0), (48711, 0), (64003, 0), (64007, 0), (64067, 0), (64071, 0), (65027, 0), (65031, 0), (65091, 0), (65095, 0)]
---------------------------
Iteration 3
Max frequency 8
Keys with max frequency [(47619, 0), (47623, 0), (47683, 0), (47687, 0), (48643, 0), (48647, 0), (48707, 0), (48711, 0), (64003, 0), (64007, 0), (64067, 0), (64071, 0), (65027, 0), (65031, 0), (65091, 0), (65095, 0)]
---------------------------
Iteration 4
Max frequency 3
Keys with max frequency [(47619, 0), (47623, 0), (47683, 0), (47687, 0), (48643, 0), (48647, 0), (48707, 0), (48711, 0), (64003, 0), (64007, 0), (64067,